In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
from torchinfo import summary
from src.network.sfcn_net import SFCNModel
from src.network.res_net import ResModel
from src.network.seres_net import SEResModel
from src.network.conv5_fc3_net import Conv5FC3Model
from src.network.vit_net import ViTModel
from src.network.transfer_net import TransferMLP

from src import config

In [15]:
sfcn = SFCNModel(config.IM_SHAPE, 40, 0.6)
summary(sfcn.encoder, config.IM_SHAPE, batch_dim=0)

Layer (type:depth-idx)                   Output Shape              Param #
SFCNEncoder                              [1, 64, 5, 6, 5]          --
├─Sequential: 1-1                        [1, 64, 5, 6, 5]          --
│    └─SFCNBlock: 2-1                    [1, 32, 80, 96, 80]       --
│    │    └─Sequential: 3-1              [1, 32, 80, 96, 80]       960
│    └─SFCNBlock: 2-2                    [1, 64, 40, 48, 40]       --
│    │    └─Sequential: 3-2              [1, 64, 40, 48, 40]       55,488
│    └─SFCNBlock: 2-3                    [1, 128, 20, 24, 20]      --
│    │    └─Sequential: 3-3              [1, 128, 20, 24, 20]      221,568
│    └─SFCNBlock: 2-4                    [1, 256, 10, 12, 10]      --
│    │    └─Sequential: 3-4              [1, 256, 10, 12, 10]      885,504
│    └─SFCNBlock: 2-5                    [1, 256, 5, 6, 5]         --
│    │    └─Sequential: 3-5              [1, 256, 5, 6, 5]         1,770,240
│    └─SFCNBlock: 2-6                    [1, 64, 5, 6, 5]      

In [16]:
sfcn = SFCNModel(config.IM_SHAPE, 40, 0.6)
summary(sfcn.classifier, sfcn.encoder.latent_shape, batch_dim=0)

Layer (type:depth-idx)                   Output Shape              Param #
SFCNClassifier                           [1, 40]                   --
├─SFCNHeadBlock: 1-1                     [1, 40]                   --
│    └─AvgPool3d: 2-1                    [1, 64, 1, 1, 1]          --
│    └─Dropout: 2-2                      [1, 64, 1, 1, 1]          --
│    └─Conv3d: 2-3                       [1, 40, 1, 1, 1]          2,600
│    └─Flatten: 2-4                      [1, 40]                   --
Total params: 2,600
Trainable params: 2,600
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.04
Forward/backward pass size (MB): 0.00
Params size (MB): 0.01
Estimated Total Size (MB): 0.05

In [17]:
transfer = TransferMLP(sfcn.encoder.latent_shape, 3, num_layers=1)
summary(transfer, sfcn.encoder.latent_shape, batch_dim=0)
# sfcn.encoder.latent_shape

Layer (type:depth-idx)                   Output Shape              Param #
TransferMLP                              [1, 3]                    --
├─FlattenEncoder: 1-1                    [1, 9600]                 --
├─MLPClassifier: 1-2                     [1, 3]                    --
│    └─Sequential: 2-1                   [1, 3]                    --
│    │    └─Dropout: 3-1                 [1, 9600]                 --
│    │    └─Linear: 3-2                  [1, 10]                   96,010
│    │    └─BatchNorm1d: 3-3             [1, 10]                   20
│    │    └─ReLU: 3-4                    [1, 10]                   --
│    │    └─Linear: 3-5                  [1, 3]                    33
Total params: 96,063
Trainable params: 96,063
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.10
Input size (MB): 0.04
Forward/backward pass size (MB): 0.00
Params size (MB): 0.38
Estimated Total Size (MB): 0.42

In [7]:
conv5_fc3 = Conv5FC3Model(config.IM_SHAPE, 40, 0.6)
summary(conv5_fc3.cuda(), config.IM_SHAPE, batch_dim=1)

Layer (type:depth-idx)                   Output Shape              Param #
Conv5FC3Model                            [1, 40]                   --
├─Conv5FC3Encoder: 1-1                   [1, 128, 5, 6, 5]         --
│    └─Sequential: 2-1                   [1, 128, 5, 6, 5]         --
│    │    └─ConvBlock: 3-1               [1, 8, 80, 96, 80]        240
│    │    └─ConvBlock: 3-2               [1, 16, 40, 48, 40]       3,504
│    │    └─ConvBlock: 3-3               [1, 32, 20, 24, 20]       13,920
│    │    └─ConvBlock: 3-4               [1, 64, 10, 12, 10]       55,488
│    │    └─ConvBlock: 3-5               [1, 128, 5, 6, 5]         221,568
├─Conv5FC3Classifier: 1-2                [1, 40]                   --
│    └─Sequential: 2-2                   [1, 40]                   --
│    │    └─Flatten: 3-6                 [1, 19200]                --
│    │    └─Dropout: 3-7                 [1, 19200]                --
│    │    └─Linear: 3-8                  [1, 1300]                 2

In [9]:
res = ResModel(config.IM_SHAPE, 40, 0.6)
summary(res.cuda(), config.IM_SHAPE, batch_dim=1)

Layer (type:depth-idx)                        Output Shape              Param #
ResModel                                      [1, 40]                   --
├─ResEncoder: 1-1                             [1, 128, 5, 6, 5]         --
│    └─Sequential: 2-1                        [1, 128, 5, 6, 5]         --
│    │    └─ResConvModule: 3-1                [1, 8, 80, 96, 80]        2,008
│    │    └─ResConvModule: 3-2                [1, 16, 40, 48, 40]       10,608
│    │    └─ResConvModule: 3-3                [1, 32, 20, 24, 20]       42,208
│    │    └─ResConvModule: 3-4                [1, 64, 10, 12, 10]       168,384
│    │    └─ResConvModule: 3-5                [1, 128, 5, 6, 5]         672,640
├─ResClassifier: 1-2                          [1, 40]                   --
│    └─Sequential: 2-2                        [1, 40]                   --
│    │    └─Dropout: 3-6                      [1, 128, 5, 6, 5]         --
│    │    └─Flatten: 3-7                      [1, 19200]                --

In [10]:
seres = SEResModel(config.IM_SHAPE, 40, 0.6)
summary(seres.cuda(), config.IM_SHAPE, batch_dim=1)

Layer (type:depth-idx)                                  Output Shape              Param #
SEResModel                                              [1, 40]                   --
├─SEResEncoder: 1-1                                     [1, 128, 5, 6, 5]         --
│    └─Sequential: 2-1                                  [1, 128, 5, 6, 5]         --
│    │    └─SEResModule: 3-1                            [1, 8, 80, 96, 80]        2,084
│    │    └─SEResModule: 3-2                            [1, 16, 40, 48, 40]       10,888
│    │    └─SEResModule: 3-3                            [1, 32, 20, 24, 20]       43,280
│    │    └─SEResModule: 3-4                            [1, 64, 10, 12, 10]       172,576
│    │    └─SEResModule: 3-5                            [1, 128, 5, 6, 5]         689,216
├─SEResClassifier: 1-2                                  [1, 40]                   --
│    └─Sequential: 2-2                                  [1, 40]                   --
│    │    └─Dropout: 3-6               

In [11]:
vit = ViTModel(config.IM_SHAPE, 40, 0.6)
summary(vit.cuda(), config.IM_SHAPE, batch_dim=1)

Layer (type:depth-idx)                             Output Shape              Param #
ViTModel                                           [1, 40]                   --
├─ViTEncoder: 1-1                                  [1, 768]                  --
│    └─ViT: 2-1                                    [1, 1, 768]               768
│    │    └─PatchEmbeddingBlock: 3-1               [1, 1200, 768]            4,068,096
│    │    └─ModuleList: 3-2                        --                        85,026,816
│    │    └─LayerNorm: 3-3                         [1, 1201, 768]            1,536
├─ViTClassifier: 1-2                               [1, 40]                   --
│    └─Sequential: 2-2                             [1, 40]                   --
│    │    └─Flatten: 3-4                           [1, 768]                  --
│    │    └─Dropout: 3-5                           [1, 768]                  --
│    │    └─Linear: 3-6                            [1, 40]                   30,760
Total params

In [ ]:
transfer = TransferMLP(config.IM_SHAPE, 40, 0.6)
summary(transfer.cuda(), config.IM_SHAPE, batch_dim=1)